# Plot generator for $\mathbf{f_\lambda}$ landscape

This notebook is used for generating plots of the perturbed loss landscape $f_\lambda$ following the method proposed in $\textit{Differentiation of Blackbox Combinatorial Solvers}$ [arXiv:1912.02175].

Problems can be added by specifying the solver and the input-generation function as shown below.
The input-generation should provide:
- the config for the solver
- a two-dimensional cut of a higher dimensional $w$-landscape 
- some gradient $\frac{dL}{dy}$ of the loss with respect to $y$, for which the perturbed linearized loss landscape $f_\lambda$ should be plotted. 

For details on the provided helper functions to generate these inputs, as well as the calculations done for the plots, see utils.flamba_utils.

Implemented are solvers for GraphMatching, MultiGraphMatching, TravellingSalesman, Ranking and ShortestPath.

In [2]:
from visualization_utils import *

from lpmp_py import gm_solver
from lpmp_py import mgm_solver
from travelling_salesman import gurobi_tsp
from shortest_path import dijkstra

ImportError: attempted relative import with no known parent package

In [2]:
class GraphMatchingBBSolver(BlackboxSolverAbstract):
    """
    Graph matching solver.
    """
    @staticmethod
    def solver(inputs, edges_left, edges_right, solver_params):
        unary_costs, quadratic_costs = inputs
        m, m_q = gm_solver(unary_costs, quadratic_costs, edges_left, edges_right, solver_params, verbose=False)
        return m, m_q
     
    @staticmethod
    def gen_input(num_nodes_l, num_edges_l, seed, wn_f=10, wn_s=0, ws_f=1, ws_s=0, y_f=1, y_s=0, directed_edges=False):
        w_slice_l, y_grad_l = gen_w_and_y_grad(
            seed=seed, 
            # Choose a different n_factor, n_shift, s_factor, s_shift to specify a different cut
            params=[dict(shape=num_nodes_l, w_slice_par=dict(mode='slice_random', n_factor=wn_f, n_shift=wn_s, s_factor=ws_f, s_shift=ws_s), 
                         y_grad_par=dict(mode='hamming_random', factor=y_f, shift=y_s)),  # unary costs
                    dict(shape=num_edges_l, w_slice_par=dict(mode='const_random', n_factor=wn_f, n_shift=wn_s, s_factor=ws_f, s_shift=ws_s), 
                         y_grad_par=dict(mode='zero'))])  # quadratic costs

        edges_left, edges_right = [gen_edges(nn, ne, directed=directed_edges) for nn, ne in zip(num_nodes_l, num_edges_l)]
        solver_params = {
            'timeout': 100, 
            'primalComputationInterval': 10, 
            'maxIter': 100000,
            'graphMatchingRounding': 'mcf', 
            'graphMatchingFrankWolfeIterations': 50
        }
        solver_config = dict(edges_left=edges_left, edges_right=edges_right, solver_params=solver_params)
        return w_slice_l, y_grad_l, solver_config
    


class MultiGraphMatchingBBSolver(BlackboxSolverAbstract):
    """
    Multigraph matching solver.
    """
    @staticmethod
    def solver(inputs, edges, solver_params):
        l = len(inputs)//2
        unary_costs_l, quadratic_costs_l = inputs[:l], inputs[l:]
        m_l, m_q_l = mgm_solver(unary_costs_l, quadratic_costs_l, edges, solver_params, verbose=False)
        return m_l + m_q_l
    
    @staticmethod
    def gen_input(num_nodes_l, num_edges_l, seed, wn_f=1, wn_s=0, ws_f=1, ws_s=0, y_f=1, y_s=0, directed_edges=False):
        unary_shapes = [(i, j) for i, j in it.combinations(num_nodes_l, 2)]
        quadratic_shapes = [(i, j) for i, j in it.combinations(num_edges_l, 2)]
        
        params_unary = [dict(shape=shape, w_slice_par=dict(mode='slice_random', n_factor=wn_f, n_shift=wn_s, s_factor=ws_f, s_shift=ws_s), 
                             y_grad_par=dict(mode='hamming_random', factor=y_f, shift=y_s)) for shape in unary_shapes]
        params_quadratic = [dict(shape=shape, w_slice_par=dict(mode='const_random', n_factor=wn_f, n_shift=wn_s, s_factor=ws_f, s_shift=ws_s), 
                                 y_grad_par=dict(mode='zero')) for shape in quadratic_shapes]
        params = params_unary + params_quadratic
        
        w_slice_l, y_grad_l = gen_w_and_y_grad(seed=seed, params=params)

        edges = [gen_edges(nn, ne, directed=directed_edges) for nn, ne in zip(num_nodes_l, num_edges_l)]
        solver_params = {
           "maxIter": 20,
           "innerIteration": 10,
           "presolveIterations": 30,
           "primalCheckingTriplets": 100,
           "multigraphMatchingRoundingMethod": "MCF_PS",
           "tighten": "",
           "tightenIteration": 50,
           "tightenInterval": 20,
           "tightenConstraintsPercentage": 0.1,
           "tightenReparametrization": "uniform:0.5"
        }
        solver_config = dict(edges=edges, solver_params=solver_params)
        return w_slice_l, y_grad_l, solver_config
    
    
class RankingBBSolver(BlackboxSolverAbstract):
    """
    Ranking solver.
    """
    @staticmethod
    def ranks_normal (sequence):
        return (np.argsort(np.argsort(sequence)[::-1])+1) / float(len(sequence))

    @staticmethod
    def solver(inputs):
        sequence = inputs[0]
        s = RankingBBSolver.ranks_normal(sequence)
        return [s]
    
    @staticmethod
    def gen_input(sequence_length, seed, wn_f=1, wn_s=0, ws_f=1, ws_s=0, y_f=1, y_s=0):
        w_slice_l, y_grad_l = gen_w_and_y_grad(
            seed=seed, 
            params=[dict(shape=[sequence_length], w_slice_par=dict(mode='slice_random', n_factor=wn_f, n_shift=wn_s, s_factor=ws_f, s_shift=ws_s),
                         y_grad_par=dict(mode='random', factor=y_f, shift=y_s))])

        solver_config = dict()
        return w_slice_l, y_grad_l, solver_config


class TSPBBSolver(BlackboxSolverAbstract):
    """
    Tsp solver.
    """
    @staticmethod
    def solver(inputs):
        matrix = inputs[0]
        m = gurobi_tsp(matrix)
        return [m]
     
    @staticmethod
    def gen_input(num_nodes, seed, wn_f=1, wn_s=0, ws_f=1, ws_s=0, y_f=1, y_s=0):
        w_slice_l, y_grad_l = gen_w_and_y_grad(
            seed=seed, 
            params=[dict(shape=(num_nodes, num_nodes), w_slice_par=dict(mode='slice_random', n_factor=wn_f, n_shift=wn_s, s_factor=ws_f, s_shift=ws_s, sym=True), 
                         y_grad_par=dict(mode='hamming_random', factor=y_f, shift=y_s, sym=True))])
        
        solver_config = dict()
        return w_slice_l, y_grad_l, solver_config


class ShortestPathBBSolver(BlackboxSolverAbstract):
    """
    Shortest path solver.
    """
    @staticmethod
    def solver(inputs, neighbourhood_fn):
        matrix = inputs[0]
        m = dijkstra(matrix, neighbourhood_fn, request_transitions=False)[0]
        #print(matrix, m)
        return [m]
     
    @staticmethod
    def gen_input(num_nodes, seed, wn_f=1, wn_s=0, ws_f=1, ws_s=0, y_f=1, y_s=0):
        w_slice_l, y_grad_l = gen_w_and_y_grad(
            seed=seed, 
            params=[dict(shape=(num_nodes, num_nodes), w_slice_par=dict(mode='slice_random', n_factor=wn_f, n_shift=wn_s, s_factor=ws_f, s_shift=ws_s, pos=True), 
                         y_grad_par=dict(mode='hamming_random', factor=y_f, shift=y_s))])
        
        solver_config = dict(neighbourhood_fn="8-grid")
        return w_slice_l, y_grad_l, solver_config
    
    



Some example problems are generated in the following:

The challenge is always to find values for the $w$-cut and the $\frac{dL}{dy}$ that produce interesting results, as in producing multiple different outputs of the solver. To find these regions there are various parameters that can be changed, including shifts and factors for changing the randomized inputs to values that "fit" the solver, as well as the seed. The boundaries for the plotting region can also be changed.

First lets have a look at some nice looking plots of $f_\lambda$:

In [3]:
lambdas = [0.000001, 1, 10, 20, 40, 100, 200]

In [4]:
rk_solv = RankingBBSolver(sequence_length=10, seed=41, wn_f=30, ws_f=20, y_s=1)  # the last three arguments change the values of w and y_grad
rk_solv.plot_flambda(lambdas=lambdas, partitions=50, bounds1=(0.0, 1.0), bounds2=(0.0, 1.0))

In [6]:
tsp_solv = TSPBBSolver(num_nodes=5, seed=42, wn_f=10, ws_f=20, y_f=1)
tsp_solv.plot_flambda(lambdas=lambdas, partitions=10, bounds1=(-1.0, 1.0), bounds2=(-1.0, 1.0), show_axes=False, show_box=False)

In [6]:
gm_solv = GraphMatchingBBSolver(num_nodes_l=[5, 5], num_edges_l=[6, 6], seed=42, wn_f=10, y_f=0.1)
gm_solv.plot_flambda(lambdas=lambdas, partitions=20, bounds1=(-1.0, 1.0), bounds2=(-1.0, 1.0))

In [7]:
sp_solv = ShortestPathBBSolver(num_nodes=10, seed=410, y_f=0.01)
sp_solv.plot_flambda(lambdas=lambdas, partitions=30, bounds1=(0.0, 1.0), bounds2=(0.0, 1.0))

Use of margin: Specifying multiple margins will plot the landscape for all lambdas, for each margin. The slider controls both lambda and margin.
In the example below, we use shortest path and examine interesting behaviour around the origin. This happens, because for $w_1 = w_2 = 0$ with the provided $w$-cut (which has the shift set to zero) we get to the point were the higher-dimensional true $w$ reaches the point 0. This is a special point as for strictly positive weights, we always have a minimum of the linearized loss ($f = w \cdot \frac{df}{dy} = 0 \cdot \frac{df}{dy} = 0$) and around this point all possible values of the loss are achieved. An introduced margin can deal with this by shifting the loss landscape away from the 0-point to the actual minimum.

In [8]:
sp_solv = ShortestPathBBSolver(num_nodes=10, seed=410, y_f=0.01, ws_f=0)
sp_solv.plot_flambda(lambdas=lambdas, margins=[0, 0.5], partitions=30, bounds1=(0.0, 1.0), bounds2=(0.0, 1.0), plot_cost=True)

Finally, the multigraph solver takes a while:

In [9]:
mgm_solv = MultiGraphMatchingBBSolver(num_nodes_l=[5, 5], num_edges_l=[6, 6], seed=42, wn_f=10, y_f=0.1)
mgm_solv.plot_flambda(lambdas=lambdas, bounds1=(-1.0,1.0), bounds2=(-1.0,1.0), partitions=10)

In [10]:
#mgm_solv = MultiGraphMatchingBBSolver(num_nodes_l=[3, 3, 3], num_edges_l=[6, 6, 6], seed=42, wn_f=10, y_f=0.1)
#mgm_solv.plot_flambda(lambdas=[0.0000001, 1, 10], bounds1=(-1.0,1.0), bounds2=(-1.0,1.0), partitions=10)